In [ ]:
## Since my computer's main drive is actually on a secondary drive, all paths could not be relative and had to be directed 
## to include the right hard drive. Please modify your code as such to work with respect to the folder and computer

## Import and Methods Definitions

In [1]:
import random
import glob
import xml.etree.ElementTree as ET
import os
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
# For Sliding Window
from skimage.transform import pyramid_gaussian
from skimage import color
from imutils.object_detection import non_max_suppression
import imutils
import time

import os,cv2,keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from keras.layers import Dense
from keras import Model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.optimizers import Adam
from keras.applications.resnet50 import ResNet50

In [3]:
import random
import glob
import xml.etree.ElementTree as ET
import os
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Test method for logic
def parseXML(file):
    root = file.getroot()
    folder = root.find('folder').text
    width = root.find('size/width').text
    objectType = root.find('object/bndbox').text
    width = int(root.find('size/width').text)
    height = int(root.find('size/height').text)
    depth = int(root.find('size/depth').text)
    size = [width, height, depth]
    return folder, width, objectType, size

# Parse car images and xml from DIR
def parseXMLtoDF_Region(osDIR):
    #Setup our DF
    d = {'filename':[], 'filepath':[], 'size':[],
     'objectType':[], 'bndbox':[], 'image':[]}
    df = pd.DataFrame(data=d)
    
    os.chdir(osDIR)
    dirlist = os.listdir()
    #filenames = glob.glob("[0-99]*.xml")  # change the pattern to match your case
    filenames = glob.glob(osDIR + '\*.xml', recursive = True)
    
    for filename in filenames:
        with open(filename, 'r', encoding="utf-8") as content:
            file = ET.parse(content)
            root = file.getroot()
            
            filename = root.find('filename').text
            imgDIR = osDIR + '\\' + filename
            img = plt.imread(imgDIR)
            #img = color.rgb2gray(img) ## COMMENT OR UNCOMMENT THIS --------------------
            filepath = imgDIR
            
            width = int(root.find('size/width').text)
            height = int(root.find('size/height').text)
            depth = int(root.find('size/depth').text)
            size = [width, height, depth]
            
            objectType = root.find('object/name').text
            xmin = int(root.find('object/bndbox/xmin').text)
            xmax = int(root.find('object/bndbox/xmax').text)
            ymin = int(root.find('object/bndbox/ymin').text)
            ymax = int(root.find('object/bndbox/ymax').text)
            
            bndbox = [xmin, ymin, xmax, ymax]
            img = cv2.resize(img, carSize) #img size change

            x_alter = carSize[0]/width # compression or expansion rate in x axis (expansion in our case, because we went up in x axis, from 333 to 800)
            y_alter = carSize[1]/height # compression or expansion rate in y axis (expansion in our case, because we went up in x axis, from 500 to 800)

            #print(x_alter, y_alter)

            # compress or expand ground truth box accordinly
            bndbox[0] = int(bndbox[0] * x_alter)
            bndbox[1] = int(bndbox[1] * y_alter)
            bndbox[2] = int(bndbox[2] * x_alter)
            bndbox[3] = int(bndbox[3] * y_alter)

            

            df2 = {'filename': filename, 'filepath': filepath, 'size': size,
             'objectType': objectType, 'bndbox': bndbox, 'image': img}
            df = df.append(df2, ignore_index = True)
    return df


# Parse Logo and Xml from DIR
def parseXMLtoDF_Logo(osDIR):
    #Setup our DF
    d = {'filename':[], 'filepath':[], 'size':[],
     'objectType':[], 'bndbox':[], 'bndbox_img':[]}
    df = pd.DataFrame(data=d)
    
    os.chdir(osDIR)
    dirlist = os.listdir()
    #filenames = glob.glob("[0-99]*.xml")  # change the pattern to match your case
    filenames = glob.glob(osDIR + '\*.xml', recursive = True)
    
    for filename in filenames:
        with open(filename, 'r', encoding="utf-8") as content:
            file = ET.parse(content)
            root = file.getroot()
            
            filename = root.find('filename').text
            imgDIR = osDIR + '\\' + filename
            img = plt.imread(imgDIR)
            #img = color.rgb2gray(img) ## COMMENT OR UNCOMMENT THIS --------------------
            filepath = imgDIR
            
            width = int(root.find('size/width').text)
            height = int(root.find('size/height').text)
            depth = int(root.find('size/depth').text)
            size = [width, height, depth]
            
            objectType = root.find('object/name').text
            if(objectType == 'toyota'):
                objectType = 'Toyota'
            elif(objectType == 'Chevorlet'):
                objectType = 'Chevrolet'
            elif(objectType == 'Hyundai-2'):
                objectType = 'Hyundai'
            elif(objectType == 'vw'):
                objectType = 'Volkswagen'
            xmin = int(root.find('object/bndbox/xmin').text)
            xmax = int(root.find('object/bndbox/xmax').text)
            ymin = int(root.find('object/bndbox/ymin').text)
            ymax = int(root.find('object/bndbox/ymax').text)
            
            bndbox = [xmin, ymin, xmax, ymax]

            startX, startY, endX, endY = bndbox
            bb_img = img[int(startY):int(endY), int(startX):int(endX)]
            bb_img = cv2.resize(bb_img, trainingSize) #img size change

            df2 = {'filename': filename, 'filepath': filepath, 'size': size,
             'objectType': objectType, 'bndbox': bndbox, 'bndbox_img': bb_img}
            df = df.append(df2, ignore_index = True)
    return df

# Output for DataFrame
def displayBNDBOXinDF(df):
    counter = 0
    approxRows = int(df['bndbox_img'].count() / 5)
    #print(approxRows)
    bndbox_imgs = df['bndbox_img'].tolist()

    # Setup plot
    fig, axes = plt.subplots(approxRows, 5, figsize=(10,10))
    axes = np.ndarray.flatten(axes)

    # Plot images
    for ax, im in zip(axes, bndbox_imgs):
        ax.set_xlabel(counter)
        ax.imshow(im)
        counter += 1

In [7]:
carSize = (800,800) #what size do you want the car images to be loaded in at

In [8]:
# HOG Prediction, either SGD or LinearSVM based off code further down
def testImage(path):
    img = plt.imread(path)
    img = cv2.resize(img, trainingSize)
    plt.imshow(img)
    fd, hog_image = hog(img, orientations=9, 
                            pixels_per_cell=(8,8),cells_per_block=(2,2), 
                            block_norm= 'L2', visualize=True)
    print(le.inverse_transform(svm.predict([fd])))
    print(svm.decision_function([fd]))
    print(np.argmax(svm.decision_function([fd])[0]))
    print(svm.decision_function([fd])[0][np.argmax(svm.decision_function([fd])[0])])

# CNN Prediction using Hog Images
def testImageCNN(path):
    img = plt.imread(path)
    img = cv2.resize(img, trainingSize)
    plt.imshow(img)
    fd, hog_image = hog(img, orientations=9, 
                            pixels_per_cell=(8,8),cells_per_block=(2,2), 
                            block_norm= 'L2', visualize=True)
    #print(le.inverse_transform(np.argmax(cnn.predict(np.array([hog_image,])))))
    print(cnn.predict(np.array([hog_image,])))  #[0]
    pred = np.argmax(cnn.predict(np.array([hog_image,])))
    print(pred)
    print(le.inverse_transform([int(pred),]))
    
# Pick a random car from car DF and shows bndbox    
def testRegionImage(df):
    value = random.randint(0, df['image'].count()-1)
    print("Image number: "+str(value))
    image = df['image'].iloc[value]
    bbox = df['bndbox'].iloc[value]
    xmin, ymin, xmax, ymax = bbox
    img_ = np.copy(image)
    cv2.rectangle(img = img_, pt1 = (xmin, ymin), pt2 = (xmax, ymax), color = (0,255,0), thickness = 2)
    cv2.imshow("img", img_)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [9]:
path = r"G:\Users\Alex\Desktop\Classes\CSE 498 - Computer Vision\Project\Logo Annotation Dataset\Toyota"
carToyotaDF = parseXMLtoDF_Region(path)
testRegionImage(carToyotaDF)
carToyotaDF.info()

0.9523809523809523 1.4285714285714286
0.8695652173913043 1.309328968903437
0.8888888888888888 1.2539184952978057
0.38095238095238093 0.5079365079365079
1.25 1.6666666666666667
0.7407407407407407 0.7407407407407407
1.1428571428571428 1.7241379310344827
2.6666666666666665 4.761904761904762
1.25 1.6666666666666667
1.25 1.6666666666666667
0.9523809523809523 1.4285714285714286
1.25 1.6666666666666667
1.25 1.6666666666666667
1.25 1.6666666666666667
1.6359918200408998 2.1220159151193636
1.25 1.6666666666666667
1.0309278350515463 1.5473887814313345
1.098901098901099 1.5655577299412915
0.6666666666666666 0.8888888888888888
0.6006006006006006 0.9411764705882353
0.7407407407407407 0.7407407407407407
0.9523809523809523 1.4285714285714286
0.7707129094412332 2.1333333333333333
0.1953125 0.29261155815654716
0.35072336694432266 0.5259697567389875
0.2777777777777778 0.4444444444444444
0.6201550387596899 0.8859357696566998
0.8830022075055187 0.8547008547008547
0.6259780907668232 0.7414272474513438
1.098

In [10]:
path = r"G:\Users\Alex\Desktop\Classes\CSE 498 - Computer Vision\Project\Logo Annotation Dataset\Chevrolet"
carChevroletDF = parseXMLtoDF_Region(path)
testRegionImage(carChevroletDF)

0.9523809523809523 1.4285714285714286
0.26666666666666666 0.4
0.2671118530884808 0.4
0.390625 0.5208333333333334
1.5625 2.0833333333333335
0.963855421686747 1.3745704467353952
0.4166666666666667 0.7407407407407407
0.625 1.1111111111111112
0.4807692307692308 0.8547008547008547
1.1111111111111112 2.3529411764705883
0.4166666666666667 0.7407407407407407
1.0 1.3333333333333333
0.8163265306122449 1.2232415902140672
0.6153846153846154 0.8368200836820083
1.0 1.4760147601476015
0.4166666666666667 0.7407407407407407
0.9523809523809523 1.4285714285714286
1.0309278350515463 1.5473887814313345
0.42735042735042733 0.7407407407407407
0.6796941376380629 1.0652463382157125
0.4 0.6001500375093773
0.78125 1.0416666666666667
0.5 0.8859357696566998
0.9302325581395349 1.2718600953895072
2.816901408450704 4.519774011299435
0.8 1.2048192771084338
1.5625 2.5
0.4166666666666667 0.7407407407407407
0.4166666666666667 0.7407407407407407
0.78125 1.0416666666666667
0.4166666666666667 0.7407407407407407
0.625 0.8333

In [11]:
path = r"G:\Users\Alex\Desktop\Classes\CSE 498 - Computer Vision\Project\Logo Annotation Dataset\Benz"
carBenzDF = parseXMLtoDF_Region(path)
testRegionImage(carBenzDF)
carBenzDF.info()

0.7407407407407407 0.7407407407407407
0.8 1.0680907877169559
2.7874564459930316 4.545454545454546
0.7407407407407407 0.7407407407407407
0.7407407407407407 0.7407407407407407
1.0 0.6672226855713094
0.4166666666666667 0.6666666666666666
0.3125 0.5555555555555556
0.390625 0.5228758169934641
0.29038112522686027 0.16336532570961812
0.44969083754918493 0.6177606177606177
1.5625 2.0833333333333335
1.0309278350515463 1.5473887814313345
0.9302325581395349 1.353637901861252
1.25 1.6666666666666667
0.8 1.2048192771084338
2.6666666666666665 4.761904761904762
1.1251758087201125 2.0
0.3125 0.5555555555555556
0.20833333333333334 0.37037037037037035
0.6745362563237775 0.97442143727162
0.6153846153846154 0.9269988412514485
1.0309278350515463 1.5473887814313345
0.9411764705882353 1.2539184952978057
1.3333333333333333 1.3333333333333333
1.7167381974248928 1.3793103448275863
1.0309278350515463 1.5473887814313345
0.2777777777777778 0.4444444444444444
0.78125 1.195814648729447
0.5 0.6872852233676976
0.61538

In [12]:
path = r"G:\Users\Alex\Desktop\Classes\CSE 498 - Computer Vision\Project\Logo Annotation Dataset\Volkswagon"
carVolkswagenDF = parseXMLtoDF_Region(path)
testRegionImage(carVolkswagenDF)
carVolkswagenDF.info()

1.3605442176870748 1.8140589569160999
1.25 1.6666666666666667
1.0309278350515463 1.5473887814313345
0.7272727272727273 1.2307692307692308
0.625 0.8333333333333334
0.5333333333333333 0.8528784648187633
1.25 1.6666666666666667
0.9523809523809523 1.4285714285714286
1.0 1.3333333333333333
1.25 1.6666666666666667
1.0309278350515463 1.5473887814313345
0.8 1.2048192771084338
0.6878761822871883 1.2232415902140672
0.78125 1.171303074670571
1.0 1.3333333333333333
0.9523809523809523 1.4285714285714286
0.8888888888888888 1.3333333333333333
0.9523809523809523 1.4285714285714286
0.9523809523809523 1.4285714285714286
0.5 0.7497656982193065
0.8 1.0666666666666667
0.9523809523809523 1.4285714285714286
0.6878761822871883 1.2232415902140672
1.0 1.3333333333333333
1.0309278350515463 1.5473887814313345
0.9523809523809523 1.4285714285714286
0.5 0.6666666666666666
0.9673518742442564 1.098901098901099
0.5797101449275363 0.7393715341959335
1.0165184243964422 1.075268817204301
0.6482982171799028 0.7866273352999

In [32]:
# combine all car images of interest
combinedRegionDF = pd.concat([carBenzDF, carChevroletDF, carToyotaDF, carVolkswagenDF])

In [33]:
# extract all car images into one array
train_images = np.stack(combinedRegionDF.image.values,axis=0)
print(train_images.shape)

(210, 800, 800, 3)


In [34]:
# extract true bndbox of logo
train_bndbox = np.stack(combinedRegionDF.bndbox.values,axis=0)
print(train_bndbox.shape)

(210, 4)


## Model Loading

In [24]:
# Model 5, 1;5 ratio RCNN
# 1 true logo image: 5 false images
model_test = keras.models.load_model(r"G:\Users\Alex\Desktop\Classes\CSE 498 - Computer Vision\Project\Logo Annotation Dataset\TEST\model\model5moreImages")

In [42]:
# Model 4, 3;4 ration RCNN
model_test2 = keras.models.load_model(r"G:\Users\Alex\Desktop\Classes\CSE 498 - Computer Vision\Project\Logo Annotation Dataset\TEST\model\model4moreImages")

In [25]:
# cnnHOGwRegion3, trained on Chev, Benz, Volksagen, Toyota
cnnHOGwRegion = keras.models.load_model(r"G:\Users\Alex\Desktop\Classes\CSE 498 - Computer Vision\Project\Logo Annotation Dataset\TEST\model\cnnHOGwRegion3")

In [43]:
# combinedDFwRegion, csv associated with cnnHOGwRegion3
combinedDF = pd.read_csv(r'G:\Users\Alex\Desktop\Classes\CSE 498 - Computer Vision\Project\Dataset\combinedDFwRegion.csv')

In [44]:
combinedDF['objectType'].value_counts()

Toyota        136
Mercedes      135
Volkswagen    133
Chevrolet     132
Name: objectType, dtype: int64

In [45]:
from sklearn.preprocessing import LabelEncoder

In [46]:
# Load in previous dataset so labeleconder will be able to translate predictions
le = LabelEncoder()
combinedDF['label'] = le.fit_transform(combinedDF.objectType)

In [47]:
combinedDF['label'].value_counts()

2    136
1    135
3    133
0    132
Name: label, dtype: int64

In [48]:
# grab file names
train_image_names = np.stack(combinedRegionDF.filename.values,axis=0)
print(train_image_names.shape)

(210,)


In [49]:
# grab true logo label
train_labels = np.stack(combinedRegionDF.objectType.values,axis=0)
print(train_labels.shape)

(210,)


In [50]:
# selective search
cv2.setUseOptimized(True);
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

In [40]:
from skimage.feature import hog

## Model 5 Run - 1:5 Ratio, 4 Classes

In [41]:
correct = 0
incorrect = 0
numpredictions = 0
for i in range(92, len(train_images), 2):
    # Grab truth
    image = train_images[i]
    bndbox_true = train_bndbox[i]
    label = train_labels[i]
    filename = train_image_names[i]
    
    # Output truth
    print("Image #: " + str(i) + " " + str(filename))
    print("\tTrue Bounding Boxes: " + str(bndbox_true))
    print("\tTrue Logo Present: " + str(label))
    
    # Selective search on image
    ss.setBaseImage(image)
    ss.switchToSelectiveSearchFast()
    ssresults = ss.process()
    imout = image.copy()
    
    # Append all predictions and rois
    predictions = []
    rois = []
    
    print("Image #" + str(i) + " Predictions:")
    for e, result in enumerate(ssresults):
        # look through all possible boxes instead of jsut 2000
        x,y,w,h = result
        xmax = x+w
        ymax = y+h
        timage = imout[y:y+h,x:x+w]
        #print(timage.shape)
        resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
        img = np.expand_dims(resized, axis=0)
        out= model_test.predict(img) #predicted on the resized region of interest
        if out[0][0] > 0.70: #if passes threshold
            predictions.append(out[0][0])
            box = [x, y, xmax, ymax, (w, h)]
            rois.append(box)
    
    if(len(rois) == 0): # break statement if nothing passes threshold
        continue
        incorrect += 1
        numpredictions += 1
    
    best_pred = np.argmax(predictions) #predict on higehst confidence
    best_roi = rois[best_pred]
    x, y, xmax, ymax, (w, h) = best_roi
    
    ROI = cv2.resize(color.rgb2gray(imout[y:ymax, x:xmax]), (224,224)) #extract hog image and features
    fd, hog_image = hog(ROI, orientations=9, 
                    pixels_per_cell=(8,8), cells_per_block=(2,2), 
                    block_norm= 'L2', visualize=True)
    # fd = fd.reshape(1, -1)
    
    numpredictions += 1
    pred = np.argmax(cnnHOGwRegion.predict(np.array([hog_image,]))) #predict on the extracted hog image
    pred_english = le.inverse_transform([pred,])[0] #convert with labelencoder
    print('\t' + str(best_roi))
    print('\t' + pred_english)
    
    if(pred_english == label):
        correct += 1
    else:
        incorrect += 1
    
    
#     pred = svm.predict(fd)
#     pred_english = str(le.inverse_transform(pred)[0])
print("Total Predictions: " + str(numpredictions))
print("Correct: " + str(correct) + " \tIncorrect:" + str(incorrect))

Image #: 92 Chev_16.jpg
	True Bounding Boxes: [348 411 441 468]
	True Logo Present: Chevorlet
Image #92 Predictions:
	[362, 419, 431, 460, (69, 41)]
	Chevrolet
Image #: 94 Chev_18.jpg
	True Bounding Boxes: [359 358 451 424]
	True Logo Present: Chevorlet
Image #94 Predictions:
	[360, 367, 438, 416, (78, 49)]
	Chevrolet
Image #: 96 Chev_20.jpeg
	True Bounding Boxes: [321 442 468 511]
	True Logo Present: Chevorlet
Image #96 Predictions:
Image #: 98 Chev_22.jpg
	True Bounding Boxes: [485 370 573 437]
	True Logo Present: Chevorlet
Image #98 Predictions:
	[479, 368, 590, 436, (111, 68)]
	Chevrolet
Image #: 100 Chev_4.jpg
	True Bounding Boxes: [341 422 451 480]
	True Logo Present: Chevorlet
Image #100 Predictions:
	[341, 419, 455, 476, (114, 57)]
	Chevrolet
Image #: 102 Chev_6.jpeg
	True Bounding Boxes: [333 380 480 438]
	True Logo Present: Chevorlet
Image #102 Predictions:
	[343, 390, 471, 431, (128, 41)]
	Chevrolet
Image #: 104 Chev_8.jpg
	True Bounding Boxes: [359 465 442 520]
	True Logo P

Image #196 Predictions:
	[385, 411, 445, 484, (60, 73)]
	Volkswagen
Image #: 198 Volkswagen_68.jpg
	True Bounding Boxes: [351 394 439 508]
	True Logo Present: Volkswagen
Image #198 Predictions:
	[358, 402, 431, 500, (73, 98)]
	Volkswagen
Image #: 200 Volkswagen_70.jpeg
	True Bounding Boxes: [366 405 435 497]
	True Logo Present: Volkswagen
Image #200 Predictions:
	[233, 93, 586, 239, (353, 146)]
	Volkswagen
Image #: 202 Volkswagen_74.jpg
	True Bounding Boxes: [363 328 456 443]
	True Logo Present: Volkswagen
Image #202 Predictions:
	[372, 340, 445, 430, (73, 90)]
	Volkswagen
Image #: 204 Volkswagen_8.jpg
	True Bounding Boxes: [372 498 443 605]
	True Logo Present: Volkswagen
Image #204 Predictions:
	[383, 519, 432, 590, (49, 71)]
	Volkswagen
Image #: 206 Volkswagen_82.jpg
	True Bounding Boxes: [338 328 401 418]
	True Logo Present: Volkswagen
Image #206 Predictions:
	[342, 340, 399, 418, (57, 78)]
	Volkswagen
Image #: 208 Volkswagen_84.jpg
	True Bounding Boxes: [353 405 418 460]
	True Logo

## Model 4 Run - 3:4 Ratio, 4 Classes

In [51]:
correct = 0
incorrect = 0
numpredictions = 0
for i in range(0, len(train_images), 2):
    # Grab truth
    image = train_images[i]
    bndbox_true = train_bndbox[i]
    label = train_labels[i]
    filename = train_image_names[i]
    
    # Output truth
    print("Image #: " + str(i) + " " + str(filename))
    print("\tTrue Bounding Boxes: " + str(bndbox_true))
    print("\tTrue Logo Present: " + str(label))
    
    # Selective search on image
    ss.setBaseImage(image)
    ss.switchToSelectiveSearchFast()
    ssresults = ss.process()
    imout = image.copy()
    
    # Append all predictions and rois
    predictions = []
    rois = []
    
    print("Image #" + str(i) + " Predictions:")
    for e, result in enumerate(ssresults):
        # look through all possible boxes instead of jsut 2000
        x,y,w,h = result
        xmax = x+w
        ymax = y+h
        timage = imout[y:y+h,x:x+w]
        #print(timage.shape)
        resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
        img = np.expand_dims(resized, axis=0)
        out= model_test2.predict(img) #predicted on the resized region of interest with MODEL_TEST2 aka model4
        if out[0][0] > 0.70: #if passes threshold
            predictions.append(out[0][0])
            box = [x, y, xmax, ymax, (w, h)]
            rois.append(box)
    
    if(len(rois) == 0): # break statement if nothing passes threshold
        continue
        incorrect += 1
        numpredictions += 1
    
    best_pred = np.argmax(predictions) #predict on higehst confidence
    best_roi = rois[best_pred]
    x, y, xmax, ymax, (w, h) = best_roi
    
    ROI = cv2.resize(color.rgb2gray(imout[y:ymax, x:xmax]), (224,224)) #extract hog image and features
    fd, hog_image = hog(ROI, orientations=9, 
                    pixels_per_cell=(8,8), cells_per_block=(2,2), 
                    block_norm= 'L2', visualize=True)
    # fd = fd.reshape(1, -1)
    
    numpredictions += 1
    pred = np.argmax(cnnHOGwRegion.predict(np.array([hog_image,]))) #predict on the extracted hog image
    pred_english = le.inverse_transform([pred,])[0] #convert with labelencoder
    print('\t' + str(best_roi))
    print('\t' + pred_english)
    
    if(pred_english == label):
        correct += 1
    else:
        incorrect += 1
    
    
#     pred = svm.predict(fd)
#     pred_english = str(le.inverse_transform(pred)[0])
print("Total Predictions: " + str(numpredictions))
print("Correct: " + str(correct) + " \tIncorrect:" + str(incorrect))

Image #: 0 Benz_1.jpeg
	True Bounding Boxes: [361 468 440 543]
	True Logo Present: Mercedes
Image #0 Predictions:
	[358, 476, 427, 538, (69, 62)]
	Volkswagen
Image #: 2 Benz_11.jpeg
	True Bounding Boxes: [395 427 448 504]
	True Logo Present: Mercedes
Image #2 Predictions:
	[282, 408, 523, 531, (241, 123)]
	Toyota
Image #: 4 Benz_13.jpeg
	True Bounding Boxes: [330 482 459 622]
	True Logo Present: Mercedes
Image #4 Predictions:
	[340, 512, 447, 608, (107, 96)]
	Mercedes
Image #: 6 Benz_16.jpeg
	True Bounding Boxes: [448 430 512 532]
	True Logo Present: Mercedes
Image #6 Predictions:
	[358, 485, 508, 544, (150, 59)]
	Chevrolet
Image #: 8 Benz_18.jpeg
	True Bounding Boxes: [348 512 446 640]
	True Logo Present: Mercedes
Image #8 Predictions:
	[210, 526, 388, 640, (178, 114)]
	Chevrolet
Image #: 10 Benz_2.jpg
	True Bounding Boxes: [350 383 448 528]
	True Logo Present: Mercedes
Image #10 Predictions:
	[357, 401, 438, 510, (81, 109)]
	Mercedes
Image #: 12 Benz_21.jpeg
	True Bounding Boxes: [38

Image #100 Predictions:
	[221, 442, 374, 539, (153, 97)]
	Chevrolet
Image #: 102 Chev_6.jpeg
	True Bounding Boxes: [333 380 480 438]
	True Logo Present: Chevorlet
Image #102 Predictions:
	[224, 596, 269, 628, (45, 32)]
	Volkswagen
Image #: 104 Chev_8.jpg
	True Bounding Boxes: [359 465 442 520]
	True Logo Present: Chevorlet
Image #104 Predictions:
	[639, 565, 696, 641, (57, 76)]
	Volkswagen
Image #: 106 toyota_11.jpg
	True Bounding Boxes: [362 384 435 464]
	True Logo Present: Toyota
Image #106 Predictions:
	[128, 295, 302, 497, (174, 202)]
	Chevrolet
Image #: 108 toyota_13.jpg
	True Bounding Boxes: [373 313 438 381]
	True Logo Present: Toyota
Image #108 Predictions:
	[373, 325, 468, 375, (95, 50)]
	Toyota
Image #: 110 toyota_19.jpg
	True Bounding Boxes: [346 448 447 536]
	True Logo Present: Toyota
Image #110 Predictions:
	[368, 466, 426, 518, (58, 52)]
	Toyota
Image #: 112 toyota_20.jpg
	True Bounding Boxes: [129 377 187 444]
	True Logo Present: Toyota
Image #112 Predictions:
	[436, 439

Image #200 Predictions:
	[353, 417, 462, 518, (109, 101)]
	Volkswagen
Image #: 202 Volkswagen_74.jpg
	True Bounding Boxes: [363 328 456 443]
	True Logo Present: Volkswagen
Image #202 Predictions:
	[372, 340, 445, 430, (73, 90)]
	Volkswagen
Image #: 204 Volkswagen_8.jpg
	True Bounding Boxes: [372 498 443 605]
	True Logo Present: Volkswagen
Image #204 Predictions:
	[381, 535, 438, 595, (57, 60)]
	Volkswagen
Image #: 206 Volkswagen_82.jpg
	True Bounding Boxes: [338 328 401 418]
	True Logo Present: Volkswagen
Image #206 Predictions:
	[342, 340, 399, 418, (57, 78)]
	Volkswagen
Image #: 208 Volkswagen_84.jpg
	True Bounding Boxes: [353 405 418 460]
	True Logo Present: Volkswagen
Image #208 Predictions:
	[145, 474, 645, 567, (500, 93)]
	Toyota
Total Predictions: 105
Correct: 42 	Incorrect:63


In [ ]:
##code based on https://github.com/1297rohit/RCNN/blob/master/RCNN.ipynb